In [417]:
import pandas as pd
df_xml = pd.read_pickle('./df_telbib_alma_aff_contries.pkl')
df_xml.head()

,authors,year,bibcode,journal,programids,affiliations,countries
0,":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I....",2016,2016ApJ...822...36O,ApJ,"2012.1.00076.S,2012.1.00080.S,2012.1.00088.S,2...","[Institute for Astronomy, University of Edinbu...","[UK, Germany, UK, UK, Germany]"
1,":Klitsch, Anne:Péroux, Céline:Zwaan, Martin A....",2019,2019MNRAS.490.1220K,MNRAS,"2017.1.00466.S,2017.1.00467.S,2017.1.00471.S,2...","[European Southern Observatory, Karl-Schwarzsc...","[Germany, Germany, Germany, UK, Germany, Germa..."
2,":Liu, Daizhong:Schinnerer, E.:Groves, B.:Magne...",2019,2019ApJ...887..235L,ApJ,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Max-Planck-Institut für Astronomie, Königstuh...","[Germany, Germany, Australia, Germany, Germany..."
3,":Liu, Daizhong:Lang, P.:Magnelli, B.:Schinnere...",2019,2019ApJS..244...40L,ApJS,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Max-Planck-Institut für Astronomie, Königstuh...","[Germany, Germany, Germany, Germany, Germany, ..."
4,":Fudamoto, Yoshinobu:Oesch, P. A.:Magnelli, B....",2020,2020MNRAS.491.4724F,MNRAS,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Department of Astronomy, University of Geneva...","[Switzerland, Switzerland, Germany, Germany, G..."


# 4. Prepare a list of ALMA projects and PIs

- Go to https://almascience.nao.ac.jp/asax/
- Only select Project code, PI name and Proposal authors
- Export all table to "VOTable(XML Format)". 
- This process may be skipped if the number of ALMA projects is not increased.

In [133]:
import linecache
import xml.etree.ElementTree as ET
from lxml import etree

def checkXML(aqxml):
    try:
        tree = ET.parse(aqxml)
    except Exception as e:
        print('type:' + str(type(e)))
        print( 'args:' + str(e.args))
        linenumber = int(str(e.args).split('line')[1].split(',')[0])
        target_line = linecache.getline(aqxmlfile, linenumber)
        print('Error occurred in:', target_line)
        errorline = linecache.clearcache() 
        

def generateDF_alma(aqxml):
    #myParser = etree.XMLParser(encoding="utf-8",recover=True)
    tree = ET.parse(aqxml)#, parser=myParser)
    root = tree.getroot()
    
    taglist =['almaprojcode', 'programauthors']

    df = pd.DataFrame(columns=taglist)

    for child in root[0]:
        if 'TABLE' in child.tag :
            count = 0
            for childchild in child:
                if 'FIELD' in childchild.tag:
                    if 'Project code' in childchild.attrib['name']:
                        col_projcode = count
                    elif 'PI name' in childchild.attrib['name']:
                        col_piname = count
                    elif 'Proposal authors' in childchild.attrib['name']:
                        col_coiname = count
                    count += 1
                if 'DATA' in childchild.tag :
                    for projid in childchild[0]:
                        if projid[col_projcode].text in df[taglist[0]].tolist():
                            continue
                        else:
                            tmp = {}
                            tmp.update({taglist[0]: projid[col_projcode].text})
                            try:
                                tmp.update({taglist[1]: projid[col_piname].text+'; '+projid[col_coiname].text})
                            except TypeError:
                                tmp.update({taglist[1]: projid[col_piname].text})

                            df = df.append(tmp,ignore_index=True)
    return df


In [343]:
#aqxmlfile='observation_1591125273.xml'
aqxmlfile='observation_1591133327.xml'
checkXML(aqxmlfile)

In [135]:
df_alma_xml = generateDF_alma(aqxmlfile)
df_alma_xml.head()

,almaprojcode,programauthors
0,2011.0.00191.S,"Boley, Aaron; Shabram, Megan; Corder, Stuartt;..."
1,2011.0.00101.S,"Wang, Wei-Hao; Huang, Kuiyun; Chen, Hsiao-Wen;"
2,2011.0.00131.S,"Maercker, Matthias; Ramstedt, Sofia; Paladini,..."
3,2011.0.00397.S,"Lonsdale, Carol; Kim, Minjin; Blain, Andrew; T..."
4,2011.0.00510.S,"Sahai, Raghvendra; Nyman, Lars-Ake; Vlemmings,..."


In [33]:
from time import gmtime, strftime

df_alma_xml.head()
today = strftime('%Y-%m-%d', gmtime())
df_alma_xml.to_pickle('./{:}_df_alma_xml.pkl'.format(today))
print('./{:}_df_alma_xml.pkl is generated'.format(today))


./2020-06-02_df_alma_xml.pkl is generated


## 5. Prepare a list of one-to-one correspondence between countries and regions

In [344]:
countrieslist = set([item for sublist in df_xml['countries'].tolist() for item in sublist])

region_list = {'-':'-',
               'Belgium': 'EU',
               'Bulgaria': 'EU',
               'Cyprus' : 'Europa+Africa',
               'Greece' : 'Europa+Africa',
               'Australia': 'Oceania+Asia',
               'South Africa': 'Europa+Africa',
               'Deceased':'Decseased',
               'Taiwan' : 'EA',
               'Norway' :  'Europa+Africa',
               'Poland' : 'EU',
               'Austria': 'EU',
               'Mexico' : 'America',
               'Germany': 'EU',
               'UK'     : 'EU',
               'Denmark': 'EU',
               'Malaysia': 'Oceania+Asia',
               'Unknown' : 'Unknown',
               'Tunisia' : 'Europa+Africa',
               'Spain' : 'EU',
               'Iceland': 'Europa+Africa',
               'Turkey': 'Oceania+Asia',
               'Vietnam': 'Oceania+Asia',
               'Ukraine': 'Europa+Africa',
               'Colombia': 'America',
               'Hungary': 'Europa+Africa',
               'Nigeria': 'Europa+Africa',
               'Israel': 'Oceania+Asia',
               'Netherlands' : 'EU',
               'Serbia': 'Europa+Africa',
               'India':  'Oceania+Asia',
               'Korea' : 'EA',
               'Collaboration': 'Collaboration',
               'Chile': 'Chile',
               'New Zealand': 'Oceania+Asia',
               'Indonesia': 'Oceania+Asia',
               'Thailand': 'Oceania+Asia',
               'France' : 'EU',
               'Brazil': 'America',
               'Finland': 'Europa+Africa',
               'United Arab Emirates': 'Oceania+Asia',
               'Canada' : 'NA',
               'Saudi Arabia': 'Oceania+Asia',
               'Hong Kong': 'Oceania+Asia',
               'Portugal': 'EU',
               'Czech Republic': 'EU',
               'Japan' : 'EA',
               'Slovenia': 'Europa+Africa',
               'Sweden' : 'EU',
               'Switzerland' :'EU',
               'Ethiopia': 'Europa+Africa',
               'USA' : 'NA',
               'China': 'Oceania+Asia',
               'Ireland':'EU',
               'Argentina' : 'America',
               'Italy' : 'EU',
               'Croatia': 'Europa+Africa',
               'Russia' : 'Europa+Africa',
               'UAE': 'Oceania+Asia',
               'Armenia': 'Oceania+Asia',
               'Crimea': 'Oceania+Asia',
               'Georgia': 'Oceania+Asia',
               'Kazakhstan': 'Oceania+Asia',
               'Uzbekistan': 'Oceania+Asia'}

appendix_df = pd.DataFrame.from_dict(region_list,orient='index')
appendix_df.to_pickle('./df_region.pkl')
appendix_df = pd.read_pickle('./df_region.pkl')
print('EA\n',appendix_df.index[appendix_df[0]=='EA'])
print('EU\n',appendix_df.index[appendix_df[0]=='EU'])
print('NA\n',appendix_df.index[appendix_df[0]=='NA'])

EA
 Index(['Taiwan', 'Korea', 'Japan'], dtype='object')
EU
 Index(['Belgium', 'Bulgaria', 'Poland', 'Austria', 'Germany', 'UK', 'Denmark',
       'Spain', 'Netherlands', 'France', 'Portugal', 'Czech Republic',
       'Sweden', 'Switzerland', 'Ireland', 'Italy'],
      dtype='object')
NA
 Index(['Canada', 'USA'], dtype='object')


# 6. Matching

In [398]:
ptn = r'(?P<almaprojcode>\d+\..\.\d+\.\w+)\_?'
df_xml_almaprojcode = df_xml[["bibcode","programids"]].set_index('bibcode').programids.str.extractall(ptn).join(df_alma_xml.set_index('almaprojcode'), on='almaprojcode').reset_index(0).reset_index(drop=True)
df_xml_almaprojcode = df_xml_almaprojcode.merge(df_xml[["bibcode","authors"]], on='bibcode')
df_xml_almaprojcode.head()


,bibcode,almaprojcode,programauthors,authors
0,2016ApJ...822...36O,2012.1.00076.S,"Scott, Kimberly; Scoville, Nick; Sanders, Davi...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
1,2016ApJ...822...36O,2012.1.00080.S,"Tsuboi, Masato; Kitamura, Yoshimi; Morimitsu, ...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
2,2016ApJ...822...36O,2012.1.00088.S,"Ouchi, Masami; Hashimoto, Takuya ; Nakajima, K...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
3,2016ApJ...822...36O,2012.1.00090.S,"Smail, Ian; Blain, Andrew; Geach, James; van K...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
4,2016ApJ...822...36O,2012.1.00123.S,"Plambeck, Richard; wright, melvyn;",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."


In [399]:

# set programauthors="archive" if projectcode == *.CAL
modindex = df_xml_almaprojcode[df_xml_almaprojcode.almaprojcode.str.count('CAL')>0].index
df_xml_almaprojcode.loc[modindex, ['programauthors']]='archive'

# set projectcode.replace("E","SV") if projectcode == *.E
modindex = df_xml_almaprojcode[df_xml_almaprojcode.almaprojcode.str.count('.E')>0].index
df_xml_almaprojcode.loc[modindex, ['almaprojcode']] = df_xml_almaprojcode.loc[modindex].almaprojcode.str.replace('.E','.SV')

# set programauthors="archive" if projectcode == *.SV
modindex = df_xml_almaprojcode[df_xml_almaprojcode.almaprojcode.str.count('SV')>0].index
df_xml_almaprojcode.loc[modindex, ['programauthors']]='archive'

df_droped = df_xml_almaprojcode[df_xml_almaprojcode.programauthors.isna()]
df_droped.to_pickle('./df_droped.pkl')
df_droped.head()

df_xml_almaprojcode_dropna = df_xml_almaprojcode.dropna()

In [400]:
df_xml_almaprojcode_dropna.head()

,bibcode,almaprojcode,programauthors,authors
0,2016ApJ...822...36O,2012.1.00076.S,"Scott, Kimberly; Scoville, Nick; Sanders, Davi...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
1,2016ApJ...822...36O,2012.1.00080.S,"Tsuboi, Masato; Kitamura, Yoshimi; Morimitsu, ...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
2,2016ApJ...822...36O,2012.1.00088.S,"Ouchi, Masami; Hashimoto, Takuya ; Nakajima, K...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
3,2016ApJ...822...36O,2012.1.00090.S,"Smail, Ian; Blain, Andrew; Geach, James; van K...",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."
4,2016ApJ...822...36O,2012.1.00123.S,"Plambeck, Richard; wright, melvyn;",":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I...."


In [401]:
df_xml_almaprojcode_dropna["PI"] = df_xml_almaprojcode_dropna.authors.str.split(':').apply(set)&df_xml_almaprojcode_dropna.programauthors.str.split(';').apply(set)

/Users/rmiura/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [402]:
df_xml_almaprojcode_dropna[df_xml_almaprojcode_dropna.PI==False]

,bibcode,almaprojcode,programauthors,authors,PI
306,2019MNRAS.490.1220K,2017.1.01188.S,"Yamaguchi, Masaki; null",":Klitsch, Anne:Péroux, Céline:Zwaan, Martin A....",False
336,2019MNRAS.490.1220K,2017.1.01306.S,"Sliwa, Kazimierz; null",":Klitsch, Anne:Péroux, Céline:Zwaan, Martin A....",False
451,2019MNRAS.490.1220K,2017.1.01694.S,"Oteo, Ivan; null",":Klitsch, Anne:Péroux, Céline:Zwaan, Martin A....",False
957,2016PASJ...68....6A,2011.0.00526.S,"Carpenter, John; null",":Ando, Ryo:Kohno, Kotaro:Tamura, Yoichi:Izumi,...",False
960,2019ApJ...872..112V,2011.0.00015.SV,archive,":van der Marel, Nienke:Dong, Ruobing:di France...",False
...,...,...,...,...,...
3791,2018ApJ...856...72O,2013.A.00014.S,"Ivison, Rob; null",":Oteo, I.:Ivison, R. J.:Dunne, L.:Manilla-Robl...",False
3793,2014A&A...562A..96I,2011.0.00008.SV,archive,":Israel, F. P.:Güsten, R.:Meijerink, R.:Loenen...",False
3811,2018A&A...619A.171B,2011.0.00526.S,"Carpenter, John; null",":Benisty, M.:Juhász, A.:Facchini, S.:Pinilla, ...",False
3823,2017MNRAS.464.4624S,2011.0.00001.SV,archive,":Sánchez-Ramírez, R.:Hancock, P. J.:Jóhannesso...",False


In [403]:
df_xml = df_xml.join(df_xml_almaprojcode_dropna.groupby('bibcode').PI.apply(list), on='bibcode')


In [418]:
def addarchive(df_master,df):
    ptn = r'(?P<almaprojcode>\d+\..\.\d+\.\w+)\_?'
    df_master = df[["bibcode","programids"]].set_index('bibcode').programids.str.extractall(ptn).join(df_alma_xml.set_index('almaprojcode'), on='almaprojcode').reset_index(0).reset_index(drop=True)
    df_master = df_master.merge(df[["bibcode","authors"]], on='bibcode')

    # set programauthors="archive" if projectcode == *.CAL
    modindex = df_master[df_master.almaprojcode.str.count('CAL')>0].index
    df_master.loc[modindex, ['programauthors']]='archive'

    # set projectcode.replace("E","SV") if projectcode == *.E
    modindex = df_master[df_master.almaprojcode.str.count('.E')>0].index
    df_master.loc[modindex, ['almaprojcode']] = df_master.loc[modindex].almaprojcode.str.replace('.E','.SV')

    # set programauthors="archive" if projectcode == *.SV
    modindex = df_master[df_master.almaprojcode.str.count('SV')>0].index
    df_master.loc[modindex, ['programauthors']]='archive'

    df_droped = df_master[df_master.programauthors.isna()]
    df_droped.to_pickle('./df_droped.pkl')
    #df_droped.head()

    df_master_dropna = df_master.dropna()

    df_master_dropna["PI"] = df_master_dropna.authors.str.split(':').apply(set)&\
                             df_master_dropna.programauthors.str.split(';').apply(set)
    
    return df.join(df_master_dropna.groupby('bibcode').PI.apply(list), on='bibcode')



def addPIorArchive(df):

    def func(x):
        if all(x):
            return "PI"
        elif any(x):
            return "PI+archive"
        else:
            return "archive"
        
    return df.PI.apply(func)

def addPIRegion(df,dct=region_list):
    return [dct[i[0]] for i in df.countries]



In [419]:
df_xml = addarchive(df_xml_almaprojcode,df_xml)
df_xml['archivedata'] = addPIorArchive(df_xml)
df_xml['region'] = addPIRegion(df_xml,dct=region_list)

/Users/rmiura/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [423]:
if set(df_xml['archivedata'].to_list()) != {'PI', 'PI+archive', 'archive'}:
    print('Invalid value(s) exists in the "archivedata" column.')
    

print('Confirmed that "region" values are all available.',set(df_xml['region'].to_list()))

df_xml.to_pickle('./{:}_telbib_alma_aff_countires_archive.pkl'.format(today))
print('./{:}_telbib_alma_aff_countires_archive.pkl is generated'.format(today))
#df_xml = pd.read_pickle('./2019-07-17_telbib_alma_aff_countires_archive.pkl')
df_xml.head(n=5)

Confirmed that "region" values are all available. {'EA', 'Collaboration', 'NA', 'Europa+Africa', '-', 'America', 'Oceania+Asia', 'Chile', 'EU'}
./2020-06-02_telbib_alma_aff_countires_archive.pkl is generated


,authors,year,bibcode,journal,programids,affiliations,countries,PI,archivedata,region
0,":Oteo, I.:Zwaan, M. A.:Ivison, R. J.:Smail, I....",2016,2016ApJ...822...36O,ApJ,"2012.1.00076.S,2012.1.00080.S,2012.1.00088.S,2...","[Institute for Astronomy, University of Edinbu...","[UK, Germany, UK, UK, Germany]","[True, True, True, True, True, True, True, Tru...",PI,EU
1,":Klitsch, Anne:Péroux, Céline:Zwaan, Martin A....",2019,2019MNRAS.490.1220K,MNRAS,"2017.1.00466.S,2017.1.00467.S,2017.1.00471.S,2...","[European Southern Observatory, Karl-Schwarzsc...","[Germany, Germany, Germany, UK, Germany, Germa...","[True, True, True, True, True, True, True, Tru...",PI+archive,EU
2,":Liu, Daizhong:Schinnerer, E.:Groves, B.:Magne...",2019,2019ApJ...887..235L,ApJ,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Max-Planck-Institut für Astronomie, Königstuh...","[Germany, Germany, Australia, Germany, Germany...","[True, True, True, True, True, True, True, Tru...",PI,EU
3,":Liu, Daizhong:Lang, P.:Magnelli, B.:Schinnere...",2019,2019ApJS..244...40L,ApJS,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Max-Planck-Institut für Astronomie, Königstuh...","[Germany, Germany, Germany, Germany, Germany, ...","[True, True, True, True, True, True, True, Tru...",PI,EU
4,":Fudamoto, Yoshinobu:Oesch, P. A.:Magnelli, B....",2020,2020MNRAS.491.4724F,MNRAS,"2011.0.00064.S,2011.0.00097.S,2011.0.00539.S,2...","[Department of Astronomy, University of Geneva...","[Switzerland, Switzerland, Germany, Germany, G...","[True, True, True, True, True, True, True, Tru...",PI,EU
